<a href="https://colab.research.google.com/github/princesinha24/Liver_class/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing all necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
from torchsummary import summary
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#defining device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# Define AlexNet network structure
class Alex(nn.Module):
    def __init__(self):
        super(Alex, self).__init__()
        self.layer1 = nn.Sequential( 
            nn.Conv2d(3, 96, kernel_size=11, stride=4), # 96*54*54
            nn.MaxPool2d(kernel_size=3, stride=2), # 96*27*27
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(num_features= 96),
            )
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, padding=2), # 256*27*27
            nn.MaxPool2d(kernel_size=3, stride=2), # 256*13*13
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(num_features= 256),
            )
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, padding=1), # 384*13*13
            )
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, padding=1), # 384*13*13
            )
 
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, padding=1), # 256*13*13
            nn.MaxPool2d(kernel_size=3, stride=2), # 256*6*6
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(num_features= 256),
            )
        self.fc1 = nn.Linear(256*5*5, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 4)
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(-1, 256*5*5)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [ ]:
#summary of network
model = Alex().to(device)
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          34,944
         MaxPool2d-2           [-1, 96, 26, 26]               0
              ReLU-3           [-1, 96, 26, 26]               0
       BatchNorm2d-4           [-1, 96, 26, 26]             192
            Conv2d-5          [-1, 256, 26, 26]         614,656
         MaxPool2d-6          [-1, 256, 12, 12]               0
              ReLU-7          [-1, 256, 12, 12]               0
       BatchNorm2d-8          [-1, 256, 12, 12]             512
            Conv2d-9          [-1, 384, 12, 12]         885,120
           Conv2d-10          [-1, 384, 12, 12]       1,327,488
           Conv2d-11          [-1, 256, 12, 12]         884,992
        MaxPool2d-12            [-1, 256, 5, 5]               0
             ReLU-13            [-1, 256, 5, 5]               0
      BatchNorm2d-14            [-1, 25

In [ ]:
!pip install pkbar
import pkbar

In [ ]:
batch_size= 32
device = 'cuda'
num_classes= 4
input_shape=(3,224,224)
learning_rate=0.001
num_epochs= 30
load_model=True

In [ ]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
main_path= 'drive/MyDrive/KMC Dataset'
train_dir= os.path.join(main_path,'Training')
test_dir= os.path.join(main_path,'Test')
val_dir= os.path.join(main_path,'Validation')

check_pt_file= os.path.join(main_path, 'AlexNet.pth.tar')

print(os.listdir(main_path))
print(os.listdir(train_dir))
print(os.listdir(test_dir))
print(os.listdir(val_dir))

['AlexNet.ipynb', 'Training', 'Validation', 'Test', 'ResNet50_Checkpoint_1.pth.tar', 'ResNet50.ipynb', 'DenseNet.ipynb', 'AlexNet_checkpoint.pth.tar']
['grade1', 'grade3', 'grade2', 'grade0']
['grade1', 'grade0', 'grade2', 'grade3']
['grade2', 'grade1', 'grade0', 'grade3']


In [ ]:
train_data= datasets.ImageFolder(train_dir, transform= transforms.ToTensor())
test_data= datasets.ImageFolder(test_dir, transform= transforms.ToTensor())
val_data= datasets.ImageFolder(val_dir, transform= transforms.ToTensor())

In [ ]:
train_loader= DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader= DataLoader(test_data, batch_size=batch_size, shuffle=True)
val_loader= DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [ ]:
train_data.class_to_idx

{'grade0': 0, 'grade1': 1, 'grade2': 2, 'grade3': 3}

In [ ]:
def save_checkpoint(state,filename="drive/MyDrive/KMC Dataset/AlexNet_checkpoint.pth.tar"):
  torch.save(state,filename)
  print("save checkpoint")
def load_checkpoint(checkpoint):
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  print("load check point")

In [ ]:
if load_model:
  load_checkpoint(torch.load("drive/MyDrive/KMC Dataset/AlexNet_checkpoint.pth.tar"))

load check point


In [ ]:
# Train Network
for epoch in range(num_epochs):
    x=0
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        x+=loss
        #print(batch_idx,loss)
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    print('epoch no',epoch,"loss",x/batch_idx)

epoch no 0 loss tensor(0.1694, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 1 loss tensor(1.3053, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 2 loss tensor(4.0529, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 3 loss tensor(0.5844, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 4 loss tensor(0.4886, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 5 loss tensor(0.8866, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 6 loss tensor(0.3036, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 7 loss tensor(0.4932, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 8 loss tensor(0.1193, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 9 loss tensor(0.0836, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 10 loss tensor(1.1098, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 11 loss tensor(0.2250, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 12 loss tensor(0.5722, device='cuda:0', grad_fn=<DivBackward0>)
epoch no 13 loss tensor(0.2031, device='cuda:0', grad_fn=<Div

In [ ]:
checkpoint={'state_dict':model.state_dict(),'optimizer':optimizer.state_dict()}
save_checkpoint(checkpoint)

save checkpoint


In [ ]:
#check accuracy and create confussion matrix
def check_accuracy(loader, model):
    #confusion matrix
    con=np.zeros([4,4],dtype=np.long)
    print("Checking accuracy on data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            #print (predictions,y)
            for i in range(y.size(0)):
              con[predictions[i]][y[i]]+=1
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print (con)
        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()
    return con

In [ ]:
con_train=check_accuracy(train_loader, model)
con_test=check_accuracy(test_loader, model)

Checking accuracy on data
[[672   0   0   0]
 [  0 610   0   0]
 [  0  30 670   0]
 [  0   0   0 665]]
Got 2617 / 2647 with accuracy 98.87
Checking accuracy on data
[[69  0  0  0]
 [ 0 65  4  2]
 [ 1 15 76  8]
 [ 0  0  0 40]]
Got 250 / 280 with accuracy 89.29


In [ ]:
def precision_recall_f1(confusion_matrix):
  num_classes= confusion_matrix.shape[0]
  precision= np.zeros(num_classes, np.float64)
  recall= np.zeros(num_classes, np.float64)
  f1= np.zeros(num_classes, np.float64)

  for i in range(num_classes):
    precision[i]= confusion_matrix[i,i]/np.sum(confusion_matrix[i])
    recall[i]= confusion_matrix[i,i]/np.sum(confusion_matrix[:,i])
    f1[i]= 2*precision[i]*recall[i]/(precision[i]+recall[i])

  return precision, recall, f1

In [ ]:
precision, recall, f1= precision_recall_f1(con_test)
print(precision,recall,f1,sep='\n')

[1.         0.91549296 0.76       1.        ]
[0.98571429 0.8125     0.95       0.8       ]
[0.99280576 0.86092715 0.84444444 0.88888889]


In [ ]:
print("Precision:",np.mean(precision))
print("Recall:",np.mean(recall))
print("F1 score:",np.mean(f1))

Precision: 0.9188732394366197
Recall: 0.8870535714285714
F1 score: 0.8967665602617244


In [ ]:
iterator2= iter(test_loader)
t2 = next(iterator2)
images, labels= t2

model.eval()
scores= model(images.to(device=device))

plt.figure(figsize=(20,5))
for r in range(0,20):
    plt.subplot(2,10,r+1)
    f= plt.imshow(images[r,0],cmap='gray')
    plt.title(int((scores.max(1))[1][r]))
    print('Expected',int(labels[r]),'Got',int((scores.max(1))[1][r]),end=' | ')
plt.show()

Expected 2 Got 2 | Expected 1 Got 1 | Expected 1 Got 1 | Expected 1 Got 1 | Expected 0 Got 0 | Expected 2 Got 1 | Expected 2 Got 2 | Expected 0 Got 0 | Expected 2 Got 2 | Expected 0 Got 0 | Expected 2 Got 2 | Expected 0 Got 0 | Expected 1 Got 1 | Expected 0 Got 0 | Expected 2 Got 2 | Expected 0 Got 0 | Expected 2 Got 2 | Expected 0 Got 0 | Expected 0 Got 0 | Expected 3 Got 3 | 